In [1]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo and Coin-OR solvers.
#for reference, see https://jckantor.github.io/ND-Pyomo-Cookbook/notebooks/01.02-Running-Pyomo-on-Google-Colab.html#installing-pyomo-and-solvers

%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

from pyomo.environ import *

In [ ]:
import pyomo.environ as pyo
import pandas as pd

# Load and preprocess data
file_path = 'ev_charging_patterns.csv'
ev_data = pd.read_csv(file_path)

# Aggregate demand by location
demand_data = ev_data.groupby('Charging Station Location').agg(
    demand=('Energy Consumed (kWh)', 'sum'),
    sessions=('Charging Station ID', 'count')
).reset_index()

In [ ]:
# Define candidate sites
candidate_sites = ev_data['Charging Station ID'].unique()
installation_costs = {site: 50000 for site in candidate_sites}  # Example cost per site
capacity = {site: 1000 for site in candidate_sites}  # Example capacity (kWh)

# Create a simplified distance matrix (binary proximity: 1 if close enough, else 0)
distance_threshold = 100  # Example threshold
locations = ev_data[['Charging Station ID', 'Charging Station Location']].drop_duplicates()
distances = {
    (i, j): 1 if i != j else 0  # Example proximity logic
    for i in candidate_sites
    for j in demand_data['Charging Station Location']
}

# Fill in missing distances with a default value
all_pairs = [(i, j) for i in candidate_sites for j in demand_data['Charging Station Location']]
for pair in all_pairs:
    if pair not in distances:
        distances[pair] = 0  # Default to 0 if not provided


# Parameters
budget = 500000  # Total budget
penalty_unmet_demand = 100  # Penalty cost per unmet demand unit (kWh)

In [ ]:
# Model
model = pyo.ConcreteModel()

# Sets
model.sites = pyo.Set(initialize=candidate_sites)
model.zones = pyo.Set(initialize=demand_data['Charging Station Location'])

# Parameters
model.installation_costs = pyo.Param(model.sites, initialize=installation_costs)
model.capacity = pyo.Param(model.sites, initialize=capacity)
model.demand = pyo.Param(model.zones, initialize=demand_data.set_index('Charging Station Location')['demand'].to_dict())
model.distances = pyo.Param(model.sites, model.zones, initialize=distances, within=pyo.Binary)

# Variables
model.x = pyo.Var(model.sites, within=pyo.Binary)  # Installation decision
model.y = pyo.Var(model.sites, model.zones, within=pyo.Binary)  # Coverage decision

# Objective: Minimize installation costs and penalty for unmet demand
model.obj = pyo.Objective(
    expr=sum(model.installation_costs[i] * model.x[i] for i in model.sites) +
         sum(
             penalty_unmet_demand * (1 - sum(model.y[i, j] for i in model.sites if (i, j) in model.distances and model.distances[i, j] == 1))
             * model.demand[j]
             for j in model.zones
         ),
    sense=pyo.minimize
)

In [ ]:
# Constraints
def coverage_constraint(model, j):
    return sum(model.y[i, j] for i in model.sites if model.distances[i, j]) >= 1

model.coverage = pyo.Constraint(model.zones, rule=coverage_constraint)

def capacity_constraint(model, i):
    return sum(model.y[i, j] * model.demand[j]
               for j in model.zones
               if (i, j) in model.distances and model.distances[i, j]) <= model.capacity[i] * model.x[i]

model.capacity = pyo.Constraint(model.sites, rule=capacity_constraint)

def budget_constraint(model):
    return sum(model.installation_costs[i] * model.x[i] for i in model.sites) <= budget

model.budget = pyo.Constraint(rule=budget_constraint)

This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
ERROR:pyomo.core:Rule failed when generating expression for Constraint capacity with index Station_391:
RecursionError: maximum recursion depth exceeded
ERROR:pyomo.core:Constructing component 'capacity' from data=None failed:
    RecursionError: maximum recursion depth exceeded


RecursionError: maximum recursion depth exceeded

In [ ]:
# Solve
solver = pyo.SolverFactory('cbc')
results = solver.solve(model)

# Display results
print("Optimal Solution:")
for site in model.sites:
    if pyo.value(model.x[site]) > 0.5:
        print(f"Install station at: {site}")
        served_zones = [j for j in model.zones if pyo.value(model.y[site, j]) > 0.5]
        print(f"  Serves zones: {served_zones}")


In [ ]:
import pyomo.environ as pyo
import pandas as pd

# Load and preprocess data
file_path = 'ev_charging_patterns.csv'
ev_data = pd.read_csv(file_path)

# Aggregate demand by location
demand_data = ev_data.groupby('Charging Station Location').agg(
    demand=('Energy Consumed (kWh)', 'sum'),
    sessions=('Charging Station ID', 'count')
).reset_index()

# Define candidate sites
candidate_sites = ev_data['Charging Station ID'].unique()
installation_costs = {site: 50000 for site in candidate_sites}  # Example cost per site
capacity = {site: 1000 for site in candidate_sites}  # Example capacity (kWh)

# Create a simplified distance matrix
distance_threshold = 100  # Example threshold
locations = ev_data[['Charging Station ID', 'Charging Station Location']].drop_duplicates()

# Initialize distances
distances = {
    (i, j): 1 if i != j else 0  # Example proximity logic
    for i in candidate_sites
    for j in demand_data['Charging Station Location']
}

# Ensure all pairs have a default value
all_pairs = [(i, j) for i in candidate_sites for j in demand_data['Charging Station Location']]
for pair in all_pairs:
    if pair not in distances:
        distances[pair] = 0  # Default to 0 if not provided

# Parameters
budget = 500000  # Total budget
penalty_unmet_demand = 100  # Penalty cost per unmet demand unit (kWh)

# Model
model = pyo.ConcreteModel()

# Sets
model.sites = pyo.Set(initialize=candidate_sites)
model.zones = pyo.Set(initialize=demand_data['Charging Station Location'])

# Parameters
model.installation_costs = pyo.Param(model.sites, initialize=installation_costs)
model.capacity = pyo.Param(model.sites, initialize=capacity)
model.demand = pyo.Param(model.zones, initialize=demand_data.set_index('Charging Station Location')['demand'].to_dict())
model.distances = pyo.Param(model.sites, model.zones, initialize=distances, default=0, within=pyo.Binary)

# Variables
model.x = pyo.Var(model.sites, within=pyo.Binary)  # Installation decision
model.y = pyo.Var(model.sites, model.zones, within=pyo.Binary)  # Coverage decision

# Objective: Minimize installation costs and penalty for unmet demand
model.obj = pyo.Objective(
    expr=sum(model.installation_costs[i] * model.x[i] for i in model.sites) +
         sum(
             penalty_unmet_demand * (1 - sum(model.y[i, j] for i in model.sites if model.distances[i, j] == 1))
             * model.demand[j]
             for j in model.zones
         ),
    sense=pyo.minimize
)

# Constraints
# Coverage constraint: Each demand zone must be covered by at least one station
def coverage_constraint(model, j):
    return sum(model.y[i, j] for i in model.sites if model.distances[i, j] == 1) >= 1

model.coverage = pyo.Constraint(model.zones, rule=coverage_constraint)

# Capacity constraint: Stations can only serve within their capacity if installed
def capacity_constraint(model, i):
    # Use a pre-calculated list of relevant zones for each site to avoid recursive calls within the constraint
    relevant_zones = [j for j in model.zones if model.distances[i, j] == 1]
    return sum(model.y[i, j] * model.demand[j] for j in relevant_zones) <= model.capacity[i] * model.x[i]

model.capacity_constraint = pyo.Constraint(model.sites, rule=capacity_constraint) # Renamed constraint

# Budget constraint: Total installation costs must not exceed the budget
def budget_constraint(model):
    return sum(model.installation_costs[i] * model.x[i] for i in model.sites) <= budget

model.budget = pyo.Constraint(rule=budget_constraint)

# Solve the model
solver = pyo.SolverFactory('cbc')
results = solver.solve(model)

# Display results
print("\nOptimal Solution:")
for site in model.sites:
    if pyo.value(model.x[site]) > 0.5:
        print(f"Install station at: {site}")
        served_zones = [j for j in model.zones if pyo.value(model.y[site, j]) > 0.5]
        print(f"  Serves zones: {served_zones}")


  - termination condition: infeasible
  - message from solver: <undefined>
ERROR:pyomo.common.numeric_types:evaluating object as numeric value: x[Station_391]
    (object: <class 'pyomo.core.base.var.VarData'>)
No value for uninitialized NumericValue object x[Station_391]



Optimal Solution:


ValueError: No value for uninitialized NumericValue object x[Station_391]

In [ ]:
import pyomo.environ as pyo
import pandas as pd

# ... (your data loading and preprocessing code) ...

# Model
model = pyo.ConcreteModel()

# ... (your sets and parameters) ...

# Variables
model.x = pyo.Var(model.sites, within=pyo.Binary)  # Installation decision
model.y = pyo.Var(model.sites, model.zones, within=pyo.Binary)  # Coverage decision

# Objective: Minimize installation costs and penalty for unmet demand
model.obj = pyo.Objective(
    expr=sum(model.installation_costs[i] * model.x[i] for i in model.sites) +
         sum(
             penalty_unmet_demand * (1 - sum(model.y[i, j] for i in model.sites if model.distances[i, j] == 1))
             * model.demand[j]
             for j in model.zones
         ),
    sense=pyo.minimize
)

# Constraints
# Coverage constraint: Each demand zone must be covered by at least one station
def coverage_constraint(model, j):
    # Use a pre-calculated list of relevant sites for each zone to avoid recursive calls within the constraint
    relevant_sites = [i for i in model.sites if model.distances[i, j] == 1]
    return sum(model.y[i, j] for i in relevant_sites) >= 1

model.coverage_constraint = pyo.Constraint(model.zones, rule=coverage_constraint)  # Renamed constraint

# Capacity constraint: Stations can only serve within their capacity if installed
def capacity_constraint(model, i):
    # Use a pre-calculated list of relevant zones for each site to avoid recursive calls within the constraint
    relevant_zones = [j for j in model.zones if model.distances[i, j] == 1]
    return sum(model.y[i, j] * model.demand[j] for j in relevant_zones) <= model.capacity[i] * model.x[i]

model.capacity_constraint = pyo.Constraint(model.sites, rule=capacity_constraint) # Renamed constraint

# Budget constraint: Total installation costs must not exceed the budget
def budget_constraint(model):
    return sum(model.installation_costs[i] * model.x[i] for i in model.sites) <= budget

model.budget_constraint = pyo.Constraint(rule=budget_constraint) # Renamed constraint

# ... (your solver and results display code) ...